# Práctica 2: Preprocesado básico de datos

## Aprendizaje Automático I

En muchos problemas de Aprendizaje Automático es necesario realizar un procesado de los datos antes de aplicar los métodos de aprendizaje. Esta fase se denomina como Ingeniería de Datos y es en si misma una disciplina. Aunque será cubierta con profundidad en la asignatura Análisis Exploratorio de Datos y Visualización, hay procesos muy sencillos como dividir los datos en un conjunto de entrenamiento y otro para prueba, convertir un dato simbólico en uno numérico, o escalar los valores numéricos de un conjunto de datos a un determinado rango. Estas serán las tareas que se realizarán en esta práctica.

## Valores perdidos

En determinados problems puede ocurrir que no se disponga de todos los datos porque algunos no se hayan podido obtener, dando lugar a lo que se conoce como valores perdidos. Existen diferentes técnicas para tratarlos pero si el número no es muy alto comparado, una solución muy sencilla es eliminarlos. Hay otras técnicas denominadas de imputación cuyo objetivo es intentar estimar el valor perdido pero esta fuera del ámbito de esta asignatura.

Pandas permite la detección de los valores perdidos de forma muy sencilla ya que se detectan con la función isna(). Devuelve un dataframe con las posiciones perdidas a True y el resto a False. En el caso de pasarle una columna, devuelve una serie.

Explicar que la variable columns de un dataframe devuelve el nombre de todas las columnas.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('iris-small-perdidos.csv')

Devuelve un dataframe

In [ ]:
perdidos = df.isna()
print(perdidos)

Devuelve una serie

In [ ]:
perdidos = df['sepal length'].isna()
print(perdidos)

#### Ejercicio propuesto para hacer en clase

Contar el número de valores perdidos para cada una de las variables

Posible solución:

In [ ]:
variables = df.columns
for variable in variables:
    perdidos = df[variable].isna()
    print(f'Valores perdidos en variable {variable}: {np.sum(perdidos)}')

## Normalización de rangos

En algunos casos los datos poseen rangos muy diferentes para las diferentes variables que se consideran y para determinados modelos esto puede dar lugar a un rendimiento muy bajo de los mismos. Por ejemplo los modelos basados en distancias como el vecino más próximo o el centroide más próximo se ven muy afectados si todas las variables no tienen un rango similar. Otro caso son la redes neuronales o las máquinas de vectores soporte.

La librería sklearn dispone de un modulo de preprocesado que incluye la normalización de los rangos.

Explicar que la función describe muestra algunos estadísticos básicos de las variables numéricas.

In [ ]:
df = pd.read_csv('iris-small.csv')
df.describe()

Numpy posee método para obtener el máximo y mínimo de un array de forma global si no se indica el eje, por columnas si el eje es el 0 o po r filas si el eje es el 1

#### Ejercicio propuesto para hacer en clase

Implementar una función que acepte un array unidimensional (vector) numpy y escale los valores entre 0 y 1. Utilizar las funciones max y min de numpy.

Posible solución:

In [ ]:
def normaliza(vector):
    maximo = np.max(vector)
    minimo = np.min(vector)

    resultado = (vector - minimo) / (maximo - minimo)

    return resultado

In [ ]:
v = np.random.randint(20, size=10)
print(v)
res = normaliza(v)
print(res)

Sklearn incluye en el módulo preprocessing la clase MinMaxScaler que realiza un escalado lineal de las variables entre un valor mínimo y máximo. Por defecto esos valores están entre 0 y 1. La clase devuelve un array numpy aunque puede aceptar dataframes de pandas.

La API de sklearn para obtener los parámetros de la mayoría de los modelos y métodos que incluye se basa en llamar al método fit con el que se ajustan los parámetros del modelo. Luego mediante el método transform se aplica el modelo. Si se desea realizar los dos pasos a la vez se puede utilizar el método fit_transform.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
datos = df.values
variables = datos[:,:-1]
variables

In [ ]:
normalizador = MinMaxScaler()
normalizador.fit(variables)
res = normalizador.transform(variables)
res

In [ ]:
print(np.max(res,axis=0))
print(np.min(res,axis=0))

Cambiar los límites para escalar y utilizar fit_transform

In [ ]:
normalizador = MinMaxScaler((-2,1))
res = normalizador.fit_transform(variables)

In [ ]:
print(np.amax(res,axis=0))
print(np.amin(res,axis=0))

## Dividir entre train y test

Los modelos una vez entrenados y puestos en funcionamiento van a utilizar como entrada, información que no se utilizó en el proceso de entrenamiento. Sin embargo es necesario tener una aproximación al rendimiento del método una vez entrenado y antes de ser puesto en producción. Para simular esta situación, el conjunto de datos disponible se divide en dos subconjuntos. Una parte se utiliza para entrenar el modelo y otra para estimar cómo sería el rendimiento en caso de ser puesto en producción, ya que estos últimos datos no se utilzaron en el entrenamiento.

Dividir el conjunto iris en 40 para entrenamiento y 20 para test.

In [ ]:
# número de muestras de iris
print(f'Número de muestras: {len(df)}')

In [ ]:
train = df.iloc[:40,:]
test = df.iloc[40:,:]

In [ ]:
print(train)

In [ ]:
print(test)

El conjunto de train no tiene iris-virginica y el de test no tiene iris-setosa ni iris-versicolor por lo tanto el modelo no es capaz de aprender todas las clases.

La solución es mezclar aleatoriamente las muestras y luego tomar las 100 y las 50. 

Existen diferentes opciones para hacerlo. Si se tiene un dataframe de pandas, se dispone de la función sample que devuelve una muestra aleatoria del dataframe indicando que número de elementos o que fracción de los mismos se desea extraer. Si la fracción es 1, se obtiene el mismo dataframe reordenado aleatoriamente.

Importante si se van a hacer dos ejecuciones para comparar resultado, poner el valor de rand_state a uno fijo.

In [ ]:
df1 = df.sample(frac=1)
df1

La librería sklearn dispone del módulo model_selection que implementa la función train_test_split que realiza permite dividir un conjunto de datos en dos (train y test) indicando la proporción o número de muestras de cada uno. Además incluye otros argumentos como la semilla del generador de números aleatorios o la opción de estratificar (mantener la proporción de muestras por cada clase) el resultado. También permite indicar si se mezclan antes de la división o no (shuffle)

Esta función puede aceptar tanto un dataframe o un array de numpy. También permite pasarle varios argumentos, haciendo la misma división en todos.

Explicar el uso de random_state para generar siempre la misma partición. Muy importante para comparar resultados de diferentes modelos para no se vean afectados por las muestras de las particiones y todos los modelos para comparar utilicen siempre el mismo conjunto de datos tanto para entrenar como testear.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = np.random.randint(20, size=10)
y = np.random.randint(200, 250, size=10)
z = np.random.random(size=10)

x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=0.2)
print(x_train)
print(x_test)

print(z_train)
print(z_test)

Probar a ejecutar varias veces sin fijar el valor de random_state y fijándolo.

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123)

In [ ]:
df_test

Crear conjunto de datos muy desbalanceado para explicar la estratificación.

In [ ]:
x = np.random.randint(100,size=(100,4))
y = np.concatenate([np.zeros(90), np.ones(10)])

In [ ]:
y

Sin estratificación la muestra minoritaria puede que no esté bien representada en train y test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)
print(y_train)
print(y_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, stratify=y)
print(y_train)
print(y_test)

## Convertir de valores simbólicos a numéricos (**Solo si hay tiempo**)

Algunos modelos no admiten valores simbólicos y por tanto se deben convertir en numéricos. Una posibilidad es asignar un entero a cada valor simbólico existente.

La aproximación más sencilla es utilizar un diccionario de Python para realizar la conversión.

### Dejar como ejercicio de clase

Posible solución

In [ ]:
dic = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
for i in range(len(df)):
    df.loc[i,'class'] = dic[df.iloc[i]['class']]

Cuando el número de valores diferentes es muy grande puede ser un poco engorroso crear el diccionario de forma manual. Por eso sklearn posee una clase que realiza la función de conversión. La clase es LabelEncoder y la interfaz es similar a la de MinMaxScaler con funciones, fit, transform y fit_transform.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('iris-small.csv')

In [ ]:
conversor = LabelEncoder()
conversor.fit(df['class'])
res = conversor.transform(df['class'])
res